In [ ]:
# take the detected features from harris corner detection and use them as keypoints for SIFT
# implement SIFT from scratch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import skimage.color
import skimage.feature
import skimage.transform
import skimage.filters
import skimage.measure
import skimage.exposure


In [ ]:

# Divide the 16 x 16 window into a 4 x 4 grid of cells
def create_sift_keypoints_4x4_grid(image, keypoints):
    sift_keypoints = []
    for kp in keypoints:
        x, y = kp
        # Get the 16x16 patch around the keypoint
        patch = image[int(y)-8:int(y)+8, int(x)-8:int(x)+8]
        # Resize to 32x32
        patch = skimage.transform.resize(patch, (32, 32))
        # Divide into 4x4 grid
        grid = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                grid[i, j] = np.mean(patch[i*8:(i+1)*8, j*8:(j+1)*8])
        sift_keypoints.append(grid)
    return sift_keypoints

# Compute edge orientation (angle of the gradient - 90 ) for each pixel
def compute_gradient_orientation_and_magnitude(image, threshold=0.1):
    # Compute gradients
    sobel_x = skimage.filters.sobel_h(image)
    sobel_y = skimage.filters.sobel_v(image)
    # Compute orientation
    orientation = np.arctan2(sobel_y, sobel_x) * 180 / np.pi
    
    # Throw out weak edges (threshold gradient magnitude)
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    # Threshold
    magnitude[magnitude < threshold] = 0
    return orientation, magnitude

# Compute histogram of gradient orientations
def compute_histogram(orientation, magnitude, num_bins=8):
    # Create histogram
    hist = np.zeros(num_bins)
    bin_width = 360 / num_bins
    for i in range(len(orientation)):
        bin_index = int((orientation[i] + 180) / bin_width) % num_bins
        hist[bin_index] += magnitude[i]
    return hist


In [ ]:
# Load the image
image = io.imread('images/1.jpg')
# Convert to grayscale
gray_image = skimage.color.rgb2gray(image)
# Detect Harris corners
corners = skimage.feature.corner_harris(gray_image)
# Get coordinates of corners
coords = np.argwhere(corners > 0.01)
# Convert coordinates to keypoints
keypoints = np.array([coords[:, 1], coords[:, 0]]).T
# Convert to float32
keypoints = np.float32(keypoints)
# Convert image to float32
gray_image = np.float32(gray_image)
# Create SIFT detector from scratch

